In [1]:
from neo4j import GraphDatabase
from pyspark.sql import SparkSession
from pyspark.sql.functions import (approx_count_distinct, avg, col, count,
                                   desc, explode, from_unixtime, lit, size)
from pyspark.sql.types import (FloatType, IntegerType, StringType, StructField,
                               StructType)

# Data Preparation

## Getting Data

The Amazon dataset from https://jmcauley.ucsd.edu/data/amazon_v2 is a comprehensive collection of product reviews and metadata. It is categorized into various domains, such as electronics, books, clothing, and more. I have created a script that allows users to download specific categories by inputting the correct category name. This simplifies accessing targeted subsets of the dataset for research purposes.

```bash
#!/bin/bash

# Set the directory where the JSON files should be stored
DATA_DIR="bigdata-mini-project/data/"

# Prompt the user for the dataset name
read -p "Enter the name of the Amazon review dataset to download (e.g., Digital_Music): " DATASET_NAME

# Check if the JSON file for the specified dataset exists in the data directory
if [ ! -f "${DATA_DIR}${DATASET_NAME}.json" ]; then

  # If the file does not exist, download it from the first link
  echo "Downloading ${DATASET_NAME}.json file..."
  wget --no-check-certificate -P "${DATA_DIR}" "https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/${DATASET_NAME}.json.gz"

  # Extract the downloaded file
  echo "Extracting ${DATASET_NAME}.json file..."
  gunzip "${DATA_DIR}${DATASET_NAME}.json.gz"

fi

# Check if the meta JSON file for the specified dataset exists in the data directory
if [ ! -f "${DATA_DIR}meta_${DATASET_NAME}.json" ]; then

  # If the file does not exist, download it from the second link
  echo "Downloading meta_${DATASET_NAME}.json file..."
  wget --no-check-certificate -P "${DATA_DIR}" "https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_${DATASET_NAME}.json.gz"

  # Extract the downloaded file
  echo "Extracting meta_${DATASET_NAME}.json file..."
  gunzip "${DATA_DIR}meta_${DATASET_NAME}.json.gz"

fi

echo "Done."
```

## Review Data

In [2]:
spark = (
    SparkSession.builder.appName("bk-imp")
    .config("spark.executor.memory", "16g")
    .config("spark.executor.cores", "2")
    .config("spark.executor.instances", "5")
    .config(
        "spark.jars.packages",
        "org.neo4j:neo4j-connector-apache-spark_2.12:5.0.1_for_spark_3",
    )
    .getOrCreate()
)  # this spark session can connect to Neo4j

23/05/15 05:08:52 WARN Utils: Your hostname, workspace resolves to a loopback address: 127.0.1.1; using 11.11.1.73 instead (on interface eth0)
23/05/15 05:08:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/terrabot/bk-imp/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/terrabot/.ivy2/cache
The jars for the packages stored in: /home/terrabot/.ivy2/jars
org.neo4j#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ca642d0a-4baf-4bbf-b3cd-c7111fb616ae;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.12;5.0.1_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12_common;5.0.1 in central
	found org.neo4j.driver#neo4j-java-driver;4.4.11 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.apache.xbean#xbean-asm6-shaded;4.10 in central
	found org.neo4j#neo4j-cypher-dsl;2020.1.4 in central
	found org.apiguardian#apiguardian-api;1.1.0 in central
:: resolution report :: resolve 294ms :: artifacts dl 11ms
	:: modules in use:
	org.apache.xbean#xbean-asm6-shaded;4.10 from central in [default]
	org.apiguardian#apiguardian-api;1.1.0 from central in [default]
	org.neo4j#neo4j-connector-apache-spar

In [3]:
# Sample the JSON file
sample_df = spark.read.json("../data/Automotive.json", samplingRatio=0.001)

# Get the schema from the sampled data
schema = sample_df.schema

In [4]:
# Read the entire JSON file using the inferred schema
review_df = (
    spark.read.json("../data/Automotive.json", schema=schema)
    .dropDuplicates()
    .repartition(10)
)

In [5]:
review_df.show(2, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 asin           | B014GM6FX8                                                                                                                                                                                                                                                                           
 image          | null                                                                                                                                                                                                                                                                                 
 overall        | 5.0                                                                                           

In [6]:
# Rename the columns to snake style
review_df = (
    review_df.withColumnRenamed("reviewerID", "reviewer_id")
    .withColumnRenamed("reviewerName", "reviewer_name")
    .withColumnRenamed("overall", "rating")
    .withColumnRenamed("reviewText", "review_text")
    .withColumnRenamed("summary", "review_summary")
    .withColumnRenamed("unixReviewTime", "unix_review_time")
    .withColumnRenamed("reviewTime", "review_time")
    .withColumnRenamed("style", "style_name")
    .withColumnRenamed("asin", "product_id")
)

In [7]:
# Print the list of columns
print("Columns in the dataset:")
for col_name in review_df.columns:
    print(col_name)

# Print the number of rows
print(f"\nNumber of rows in the dataset: {review_df.count()}")

Columns in the dataset:
product_id
image
rating
review_text
review_time
reviewer_id
reviewer_name
style_name
review_summary
unix_review_time
verified
vote



Number of rows in the dataset: 7828023


## Preprocess Data

In [8]:
# Count the number of distinct reviewers, reviewed products, and total reviews
num_distinct_reviewers = review_df.agg(
    approx_count_distinct("reviewer_id")
).collect()[0][0]
num_distinct_products = review_df.agg(
    approx_count_distinct("product_id")
).collect()[0][0]
num_total_reviews = review_df.count()

# Calculate the average number of reviews per user
avg_reviews_per_user = num_total_reviews / num_distinct_reviewers

# Print the statistics
print(f"Number of unique reviewers         : {num_distinct_reviewers}")
print(f"Number of unique reviewed products : {num_distinct_products}")
print(f"Number of total reviews            : {num_total_reviews}")
print(f"Average number of reviews per user : {avg_reviews_per_user:.2f}")

23/05/15 05:10:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Number of unique reviewers         : 4079467
Number of unique reviewed products : 926020
Number of total reviews            : 7828023
Average number of reviews per user : 1.92


### Data Types Conversion

In [9]:
review_df = review_df.withColumn(
    "reviewTime_date", from_unixtime("unix_review_time")
)

### Sampling Data

+ Take top 10k popular items
+ Take 50k popular reviews

In [10]:
# sort by the count of reviews in descending order and take the top 10k reviewers
top_reviewers_df = (
    review_df.repartition(10)
    .groupBy("reviewer_id")
    .count()
    .orderBy(desc("count"))
    .limit(10000)
)

# sort by the count of reviews in descending order and take the top 50k products
top_products_df = (
    review_df.repartition(10)
    .groupBy("product_id")
    .count()
    .orderBy(desc("count"))
    .limit(50000)
)

sampled_review_df = (
    review_df.repartition(10)
    .join(top_reviewers_df, "reviewer_id", "inner")
    .join(top_products_df, "product_id", "inner")
    .drop("count")
    .dropDuplicates(["product_id", "reviewer_id"])
)

In [11]:
# Count the number of distinct reviewers, reviewed products, and total reviews
num_distinct_reviewers = sampled_review_df.agg(
    approx_count_distinct("reviewer_id")
).collect()[0][0]
num_distinct_products = sampled_review_df.agg(
    approx_count_distinct("product_id")
).collect()[0][0]
num_total_reviews = sampled_review_df.count()

# Calculate the average number of reviews per user
avg_reviews_per_user = num_total_reviews / num_distinct_reviewers

# Print the statistics
print(f"Number of unique reviewers         : {num_distinct_reviewers}")
print(f"Number of unique reviewed products : {num_distinct_products}")
print(f"Number of total reviews            : {num_total_reviews}")
print(f"Average number of reviews per user : {avg_reviews_per_user:.2f}")

23/05/15 05:12:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/15 05:12:37 WARN RowBasedKeyValueBatch: Calling spill() on

Number of unique reviewers         : 8956
Number of unique reviewed products : 40021
Number of total reviews            : 225761
Average number of reviews per user : 25.21


In [ ]:
sampled_review_df.write.parquet("../data/sampled_review_df")

# Graph Database

By running the command `docker-compose up -d` in the directory `bigdata-mini-project/docker/neo4j/` on a machine with Docker installed, we can start a Neo4j database instance in a Docker container. This will create a container based on the configuration in the `docker-compose.yml` file located in that directory.

The address of the Neo4j database has been mapped to http://remote.tqtensor.com:4747.

In [ ]:
df = sampled_review_df

In [ ]:
# Write nodes to Neo4j for products
df.select(col("product_id").alias("id")).dropDuplicates().repartition(
    8
).write.format("org.neo4j.spark.DataSource").option(
    "url", "bolt://remote.tqtensor.com:7687"
).option(
    "node.keys", "id"
).option(
    "labels", ":Product"
).mode(
    "overwrite"
).save()

# Write nodes to Neo4j for reviewers
df.select(col("reviewer_id").alias("id")).dropDuplicates().repartition(
    8
).write.format("org.neo4j.spark.DataSource").option(
    "url", "bolt://remote.tqtensor.com:7687"
).option(
    "node.keys", "id"
).option(
    "labels", ":User"
).mode(
    "overwrite"
).save()

# Write relationships to Neo4j between reviewers and products
df.repartition(8).write.format("org.neo4j.spark.DataSource").option(
    "url", "bolt://remote.tqtensor.com:7687"
).option("relationship.save.strategy", "keys").option(
    "relationship", "reviews"
).option(
    "relationship.properties", "rating"
).option(
    "relationship.source.labels", ":User"
).option(
    "relationship.source.node.keys", "reviewer_id:id"
).option(
    "relationship.target.labels", ":Product"
).option(
    "relationship.target.node.keys", "product_id:id"
).mode(
    "overwrite"
).save()